In [1]:
!pip install ray==2.41.0

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install numpy mlflow tensorflow "ray[serve,default,client]"

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import mlflow
import mlflow.tensorflow
import numpy as np

from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# -------------------
# Prepare Data
# -------------------
data = load_diabetes()
X = data.data
y = data.target.reshape(-1, 1)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# -------------------
# Define Model
# -------------------
def create_model(input_dim):
    model = keras.Sequential([
        layers.Dense(64, activation="relu", input_shape=(input_dim,)),
        layers.Dense(32, activation="relu"),
        layers.Dense(1)  # regression output
    ])
    model.compile(optimizer="adam", loss="mse", metrics=["mse"])
    return model

input_dim = X_train.shape[1]
epochs = 50
batch_size = 32

mlflow.set_experiment("Diabetes_Prediction_TensorFlow")

with mlflow.start_run():
    mlflow.log_param("epochs", epochs)
    mlflow.log_param("batch_size", batch_size)
    mlflow.log_param("optimizer", "adam")
    mlflow.log_param("loss_fn", "mse")
    mlflow.log_param("input_features", input_dim)

    model = create_model(input_dim)

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        verbose=0
    )

    # Evaluation
    loss, mse = model.evaluate(X_test, y_test, verbose=0)
    rmse = np.sqrt(mse)

    mlflow.log_metric("mse", mse)
    mlflow.log_metric("rmse", rmse)

2025/08/25 15:31:13 INFO mlflow.tracking.fluent: Experiment with name 'Diabetes_Prediction_TensorFlow' does not exist. Creating a new experiment.
2025/08/25 15:31:13 WARNING mlflow.tracking.context.registry: Encountered unexpected error during resolving tags: 'getpwuid(): uid not found: 1001'
2025/08/25 15:31:14 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|w

🏃 View run industrious-mare-919 at: http://ai-starter-kit-mlflow:5000/#/experiments/1/runs/b4024c97d9bd4a058ae7f005bf520d60
🧪 View experiment at: http://ai-starter-kit-mlflow:5000/#/experiments/1


In [4]:
import numpy as np
import mlflow.tensorflow
import tensorflow as tf
from starlette.requests import Request
from typing import Dict

from ray import serve
import ray


@serve.deployment(
    ray_actor_options={
        "runtime_env": {
            "pip": ["tensorflow"]
        },
    }
)
class TensorFlowMLflowDeployment:
    def __init__(self):
        print("Loading model...")
        self.model = model
        print("Model loaded successfully.")

    async def __call__(self, request: Request) -> Dict:
        try:
            data = await request.json()
            features = data.get("features", None)
            if features is None:
                return {"error": "Missing 'features' in request"}
            X = np.array(features).reshape(1, -1)

            # Make prediction with TensorFlow model
            prediction = self.model.predict(X).flatten().tolist()

            return {"prediction": prediction}
        except Exception as e:
            return {"error": str(e)}


# Bind and deploy
app = TensorFlowMLflowDeployment.bind()
serve.run(app, route_prefix="/predict")


2025-08-25 15:31:54,495	INFO worker.py:1514 -- Using address ray://ai-starter-kit-kuberay-head-svc:10001 set in the environment variable RAY_ADDRESS
2025-08-25 15:31:54,496	INFO client_builder.py:244 -- Passing the following kwargs to ray.init() on the server: log_to_driver
2025-08-25 15:31:56,843	WARNING utils.py:1591 -- Python patch version mismatch: The cluster was started with:
    Ray: 2.41.0
    Python: 3.11.11
This process on Ray Client was started with:
    Ray: 2.41.0
    Python: 3.11.9

(ProxyActor pid=1468) INFO 2025-08-25 08:31:59,214 proxy 10.244.0.10 -- Proxy starting on node 81c358455800f2bdc4877c485050214e6401aeba80d5f886b0b5c76b (HTTP port: 8000).
INFO 2025-08-25 15:31:59,273 serve 33 -- Started Serve in namespace "serve".
(ProxyActor pid=1468) INFO 2025-08-25 08:31:59,263 proxy 10.244.0.10 -- Got updated endpoints: {}.
(ServeController pid=1467) INFO 2025-08-25 08:31:59,381 controller 1467 -- Deploying new version of Deployment(name='TensorFlowMLflowDeployment', app='

(ServeReplica:default:TensorFlowMLflowDeployment pid=1892) Loading model...
(ServeReplica:default:TensorFlowMLflowDeployment pid=1892) Model loaded successfully.


INFO 2025-08-25 15:32:27,755 serve 33 -- Application 'default' is ready at http://127.0.0.1:8000/predict.
INFO 2025-08-25 15:32:27,763 serve 33 -- Deployed app 'default' successfully.


DeploymentHandle(deployment='TensorFlowMLflowDeployment')

(ServeReplica:default:TensorFlowMLflowDeployment pid=1892) INFO 2025-08-25 08:32:31,791 default_TensorFlowMLflowDeployment hxdk1mul 2fb49464-68f5-4bb3-a884-f57f44ab36f1 -- POST /predict 200 118.2ms


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
